#$\color{#d7bb1b}{\text{Introduction}}$

This pipeline clusters fasta files to a desired number of sequences.

###  Sources:

"CD-HIT: a fast program for clustering and comparing large sets of protein or nucleotide sequences", Weizhong Li & Adam Godzik. Bioinformatics, (2006) 22:1658-1659

"CD-HIT: accelerated for clustering the next generation sequencing data", Limin Fu, Beifang Niu, Zhengwei Zhu, Sitao Wu & Weizhong Li. Bioinformatics, (2012) 28:3150-3152

R.C. Edgar (2021) "MUSCLE v5 enables improved estimates of phylogenetic tree confidence by ensemble bootstrapping"
https://www.biorxiv.org/content/10.1101/2021.06.20.449169v1.full.pdf

#$\color{#17bb1b}{\text{Setup: G-Drive, BioConda, File}}$

In [ ]:
# Storage path setup (Enter path of the directory you want to save to)

%env STORAGE_FILE_PATH=/content/drive/MyDrive/Greene-Lab/CAASS

env: STORAGE_FILE_PATH=/content/drive/MyDrive/Greene-Lab/CAASS


In [ ]:
# Mounts Google Drive. (A pop-up window will appear to ask for permission)

from google.colab import drive
drive.mount('/content/drive')

! mkdir $STORAGE_FILE_PATH
! mkdir $STORAGE_FILE_PATH/input-files
! mkdir $STORAGE_FILE_PATH/cleaned-files
! mkdir $STORAGE_FILE_PATH/cluster-files
! mkdir $STORAGE_FILE_PATH/align-files
! mkdir $STORAGE_FILE_PATH/tree-files
! mkdir $STORAGE_FILE_PATH/ancestor-files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/input-files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/cleaned-files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/cluster-files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/align-files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/tree-files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Greene-Lab/CAASS/ancestor-files’: File exists


In [ ]:
# Installs Conda package manager.

! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

--2023-11-22 01:05:23--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  96.9MB/s    in 0.8s    

2023-11-22 01:05:24 (96.9 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37

In [ ]:
#@title File Upload
#@markdown <b>Input files should be unaligned fastas.</b>

#@markdown Auto upload allows upload of only one file at a time.
#@markdown A prompt will appear below this code block for you to select your file

auto_upload = True #@param {type:"boolean"}

if(auto_upload):
  from google.colab import files
  uploaded = files.upload()

  for fn in uploaded:
    !cp $fn $STORAGE_FILE_PATH/input-files $STORAGE_FILE_PATH/input-files
    %env INPUT=$fn
    !rm $fn

#@markdown If manual upload, upload your input files or into
#@markdown /CAASS/input-files on your drive and copy the file name below:

else:
  file_name = "" #@param {type:"string"}
  %env INPUT=$file_name

In [ ]:
! $fn

/bin/bash: line 1: fungi_orders_100+_rad51.fasta: command not found


In [ ]:
# Initialization of file names for the rest of the pipeline

ext = %env INPUT
%env CLEANED=cleaned_$ext
%env CLUSTERED=clustered_$ext
%env ALIGNED=aligned_$ext
%env ANCESTRAL=ancestral_$ext
%env STRUCTURED=stuctural_$ext

tree_ext = ext[:ext.find(".")] + ".tre"
%env TREE=tree_$tree_ext

env: CLEANED=cleaned_fungi_orders_100+_rad51.fasta
env: CLUSTERED=clustered_fungi_orders_100+_rad51.fasta
env: ALIGNED=aligned_fungi_orders_100+_rad51.fasta
env: ANCESTRAL=ancestral_fungi_orders_100+_rad51.fasta
env: STRUCTURED=stuctural_fungi_orders_100+_rad51.fasta
env: TREE=tree_fungi_orders_100+_rad51.tre


#$\color{#1bab8b}{\text{spaghetti: Clean-up}}$

In [ ]:
#@title Processing Options
# Initialization
path = %env STORAGE_FILE_PATH
input = %env INPUT
cleaned = %env CLEANED
outFile = path + "/cleaned-files/" + cleaned
inFile = path + "/input-files/" + input

# reads file into string array
text = ""
with open(inFile, "r") as readFile:
  for x in readFile:
    if bool(x.rstrip()):
      text += x
    else:
      print("empty")

lines = text.split("\n")

#@markdown Adds prefix to all sequence headers
#@markdown (useful when combining datasets with multiple proteins).
#@markdown [blank by default]
prefix = "" #@param {type: "string"}

for x in range(len(lines)):
  line = lines[x]
  try:
    if (line[0] == '>'):
      lines[x] = ">" + prefix + "_" + line[1:]
  except:
    print("endOfFile")


#@markdown Selecting delete_seqs removes sequences containing "X" or "-".
#@markdown
#@markdown Selecting clean_seqs keeps all sequences but deletes instances of "x" or "-".
delete_seqs = False #@param {type:"boolean"}
clean_seqs = False #@param {type:"boolean"}

index = 0
count = 0
new_lines = []

if (delete_seqs):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (body.find('X') == -1 and body.find('-') == -1):
      new_lines.append(header)
      new_lines.append(body)
    else:
      count += 1
  lines = new_lines
  print("deleted " + str(count) + " seqs for 'X' and '-'")

if (clean_seqs):
  for x in range(len(lines)):
    line = lines[x]
    try:
      if (line[0] != '>'):
        lines[x] = line.replace("X","").replace("-","")
    except:
      print("endOfFile")


#@markdown Selecting delete_low_q removes sequences labeled as "LOW-QUALITY" or
#@markdown are the secondary / tertiary isoforms of a given species' protein.
delete_low_q = False #@param {type:"boolean"}

index = 0
count = 0
new_lines = []

if(delete_low_q):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (header.find("X2") == -1 and header.find("X3") == -1 and header.find("LOW") == -1):
        new_lines.append(header)
        new_lines.append(body)
    else:
      count += 1

  lines = new_lines
  print("deleted " + str(count) + " low quality or secondary isoform seqs")

#@markdown Selecting lengthen_seqs lengthens human and yeast sequences. (this ensures
#@markdown they are representatives during the clustering process)

lengthen_seqs = False #@param {type:"boolean"}
index = 0
poly_A_tail = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

if(lengthen_seqs):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (header.lower().find("homo sapiens") != -1 or
        header.lower().find("saccharomyces cerevisiae") != -1):
      lines[2*x] = header
      lines[2*x+1] = body + poly_A_tail


#@markdown Selecting concat_header ensures entire headers shows up during tree labeling
concat_header = True #@param {type:"boolean"}

if(concat_header):
  for x in range(len(lines)):
    if lines[x][0] == '>':
      lines[x] = lines[x].replace(" ","-").replace(":","").replace("(","").replace(")","")


#writes to file
with open(outFile, "w") as writeFile:
  for line in lines:
    writeFile.write(line + "\n")

#$\color{#2f7fcf}{\text{cd-hit: Clustering}}$

In [ ]:
# Installs cd-hit
a = ! conda list
if (a.nlstr.find("cd-hit") == -1):
  ! conda install -c bioconda cd-hit -y
else:
  print("cd-hit already installed")

## cd-hit runtime

Parameter setting for input and output files. The optimal similarity threshold will be found based on the maximum sequence number provided

In [ ]:
MAX_CLUSTERS = 10000
MIN_THRESHOLD = 0.7
MAX_THRESHOLD = 0.98

In [ ]:
# setup to allow next step
%env THRESHOLD = 0
def run():
  ! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c $THRESHOLD

env: THRESHOLD=0


In [ ]:
# captures console output while clustering at various thresholds
%%capture c
for i in range(int(MIN_THRESHOLD*100), int(MAX_THRESHOLD*100)+1):
  a = i/100 # tests thresholds from min to max with increment of 0.01
  %env THRESHOLD = $a
  run()

In [ ]:
# checks output for cluster number and finds optimal threshold
# then overwrites output file with optimal clustering

maxi = 0
b=0
for i in range(int((MAX_THRESHOLD-MIN_THRESHOLD)*100)):
  index = c.stdout.find("THRESHOLD",b)
  b = c.stdout.find("finished",index)
  numOutput = c.stdout[b+16:b+20]
  if (int(numOutput) < MAX_CLUSTERS):
    maxi = i # finds the most sensitive threshold that generates < MAX_CLUSTERS

optimum = MIN_THRESHOLD + maxi*0.01
%env THRESHOLD = $optimum

! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c $THRESHOLD -g 1

In [ ]:
! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c 0.98 -g 1

Program: CD-HIT, V4.8.1, Mar 01 2019, 14:14:47
Command: cd-hit -i
         /content/drive/MyDrive/Greene-Lab/CAASS/cleaned-files/cleaned_Dmc1_1373.fasta
         -o
         /content/drive/MyDrive/Greene-Lab/CAASS/cluster-files/clustered_Dmc1_1373.fasta
         -c 0.98 -g 1

Started: Mon Aug  7 18:40:15 2023
                            Output                              
----------------------------------------------------------------
total seq: 1318
longest and shortest : 511 and 279
Total letters: 455797
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 1634696
Max number of word counting entries: 90421371

comparing sequences from          0  to       1318
.
     1318  finished        784  clusters

Approximated maximum memory consumption: 78M
writing new database


In [ ]:
#@title trim lengthened sequences
#@markdown if human and yeast sequences were lengthened in earlier steps,
#@markdown they need to be trimmed at this stage before alignment

#Initialization
clustered = %env CLUSTERED
clusterFile = path + "/cluster-files/" + clustered


trim_seqs = True #@param {type:"boolean"}

# reads file into string array
if(trim_seqs):
  text = ""
  with open(clusterFile, "r") as readFile:
    for x in readFile:
      if bool(x.rstrip()):
        text += x

  lines = text.split("\n")

  index = 0

  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if ((header.lower().find("sapiens") != -1 or
        header.lower().find("cerevisiae") != -1) and
        body[-3:] == 'AAA'):
      lines[2*x] = header
      lines[2*x+1] = body[:-120] # removes the 120 A's that were added

  with open(clusterFile, "w") as writeFile:
    for line in lines:
      writeFile.write(line + "\n")

#$\color{#8b6bbb}{\text{MUSCLE5: Alignment}}$

In [ ]:
# Installs MUSCLE5

a = ! conda list
if (a.nlstr.find("muscle") == -1):
  ! conda install -c bioconda muscle -y
else:
  print("muscle already installed")

In [ ]:
# EXECUTE THIS IF RUNNING from a folder that's not aligned-files
! muscle -in $STORAGE_FILE_PATH/cleaned-files/$CLEANED -out $STORAGE_FILE_PATH/align-files/$ALIGNED


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

cleaned_fungi_orders_100+_rad51 59 seqs, lengths min 333, max 481, avg 351
00:00:00     15 MB(4%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00     15 MB(4%)  Iter   1  100.00%  K-mer dist pass 2
00:00:01     31 MB(8%)  Iter   1  100.00%  Align node
00:00:01     31 MB(8%)  Iter   1  100.00%  Root alignment
00:00:01     31 MB(8%)  Iter   2  100.00%  Refine tree
00:00:01     31 MB(8%)  Iter   2  100.00%  Root alignment
00:00:01     31 MB(8%)  Iter   2  100.00%  Root alignment
00:00:01     31 MB(8%)  Iter   3  100.00%  Refine biparts
00:00:02     31 MB(8%)  Iter   4  100.00%  Refine biparts
00:00:02     31 MB(8%)  Iter   5  100.00%  Refine biparts
00:00:02     31 MB(8%)  Iter   5  100.00%  Refine biparts
00:00:02     31 MB(8%)  Iter   6  100.00%  Refine biparts
00:00:02     31 MB(8%)  Iter   7  100.00%  Refine bipart

In [ ]:
# File Setup
ext = %env CLUSTERED
%env ALIGNED= aligned_$ext


# Run Command
! muscle -in $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -out $STORAGE_FILE_PATH/align-files/$ALIGNED

#$\color{#bb5b8b}{\text{FastTree: Tree Building}}$

In [ ]:
# Installs FastTree
a = ! conda list
if (a.nlstr.find("fasttree") == -1):
  ! conda install -c bioconda fasttree -y
else:
  print("fasttree already installed")

fasttree already installed


In [ ]:
# File Setup (Tree will be generated in .tre format)
%env ALIGNED

# Run Command
! FastTree $STORAGE_FILE_PATH/align-files/$ALIGNED > $STORAGE_FILE_PATH/tree-files/$TREE

FastTree Version 2.1.10 Double precision (No SSE3)
Alignment: /content/drive/MyDrive/Greene-Lab/CAASS/align-files/aligned_fungi_orders_100+_rad51.fasta
Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
Initial topology in 0.05 seconds
Refining topology: 24 rounds ME-NNIs, 2 rounds ME-SPRs, 12 rounds ML-NNIs
Total branch-length 6.454 after 0.40 sec
ML-NNI round 1: LogLk = -16521.509 NNIs 14 max delta 5.27 Time 1.70
Switched to using 20 rate categories (CAT approximation)
Rate categories were divided by 0.915 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -14718.530 NNIs 3 max delta 3.73 Time 2.88
ML-NNI round 3: LogLk = -14718.416 NNIs 0 max delta 0.00 Time

#Extra

###Getting status updates from FireProt-ASR

In [ ]:
%env ID = dm8y77_16
ID = %env ID

env: ID=dm8y77_16


In [ ]:
import requests
import pprint as pp
site = 'https://loschmidt.chemi.muni.cz/fireprotasr/?action=calculation&job=' + ID + '&'
response = requests.get(site)
response.text
print(response.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
	<head>
		<meta http-equiv="content-type" content="text/html; charset=UTF-8">
		<link rel="shortcut icon" type="image/x-icon" href="favicon.ico" />
		<link rel="icon" type="image/x-icon" href="favicon.ico" />
		<link type="text/css" rel="stylesheet" href="FireProtASR.css">

		<title>FireProt-ASR</title>

		<script>var isomorphicDir="fireprotasr/sc/";</script>
		<script type="text/javascript" language="javascript" src="fireprotasr/fireprotasr.nocache.js"></script>
		<script src="fireprotasr/sc/initsc.js" language="javascript"></script>
		<script src="fireprotasr/sc/modules/ISC_Core.js"></script>
		<script src="fireprotasr/sc/modules/ISC_Foundation.js"></script>
		<script src="fireprotasr/sc/modules/ISC_Containers.js"></script>
		<script src="fireprotasr/sc/modules/ISC_Grids.js"></script>
		<script src="fireprotasr/sc/modules/ISC_Forms.js"></script>
		<script src="fireprotasr/sc/modules/ISC_RichTextEditor.js"></script

## post processing

In [ ]:
import os
try:
  os.remove("/content/drive/MyDrive/cd-hit-trial/output-files/Copy of 300_clustered_Dmc1.fasta")
except:
  print("error while deleting file.")

error while deleting file.


In [ ]:
import math

EPSILON = 1e-9

def nlogn(c):
  max=c
  min=0

  while min < max:
    mid = min + ( (max-min) / 2)
    #print(mid)

    result = mid * math.log(mid)/math.log(2)
    #print("max=" + str(max))
    #print("min=" + str(min))
    #print("result=" + str(result))

    # 3600 > nlogn: shift n up
    # 3600 < nlogn: shift n down

    if abs(c - result) < EPSILON:
      return mid
    else:
      if c - result < EPSILON:
        max = mid
      if c - result > EPSILON:
        min = mid

def runTest(c):

  i=1
  while i<c:

    if 7*(i**3) < (60* i * math.log(i)/math.log(2)):
      print(i)

    i +=1



In [ ]:
i = 10
print(7*(i**3))
print(60* i * (math.log(i)/math.log(2)))
print(7*(i**3) < (60* i * (math.log(i)/math.log(2))))

7000
1993.1568569324177
False


In [ ]:
runTest(10000)

2
3
4


In [ ]:
print(nlogn(7200000))

387825.54611194285
